In [1]:
cd ../../nkono/IVC_MDE

/projectnb/cs585bp/nkono/IVC_MDE


In [2]:
import os
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from matplotlib import pyplot as plt

from torch.utils.data import Dataset
from PIL import Image
import glob

from torchvision import models
import tqdm

import time
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision.transforms import Resize, Compose, ToPILImage, ToTensor
import pickle
import math
import random
from torchvision.transforms import RandomCrop
#os.chdir('/projectnb/cs585bp/jkoh/')
#from SILog import SILogLoss
#os.chdir('/projectnb/cs585bp/krishna/project/')
from SILog import SILogLoss
OUT_SIZE = (2200, 1550)

In [3]:
import torch

In [4]:
def neighbor_match(sam):
  """
  roll axes 8 times, and stack
  then truth test with original
  """ 
  
    #pad out the image so the indexing works
  padding = (1, 1, 1, 1, 0, 0)  # (padding_left, padding_right, padding_top, padding_bottom, padding_front, padding_back)
  #print(sam.shape)
  sam = F.pad(sam, padding, mode='constant', value=-1) #negative 1 so it never matches. 
  #print(sam.shape)
    
  surrounding_pixels = torch.cat([
    torch.roll(sam, shifts=(-1, -1), dims=(2, 3))  
    ,torch.roll(sam, shifts=(-1, 0), dims=(2, 3)) 
    ,torch.roll(sam, shifts=(-1, 1), dims=(2, 3))  

    ,torch.roll(sam, shifts=(0, -1), dims=(2, 3))  
    ,torch.roll(sam, shifts=(0, 1), dims=(2, 3))  

    ,torch.roll(sam, shifts=(1, -1), dims=(2, 3))  
    ,torch.roll(sam, shifts=(1, 0), dims=(2, 3))  
    ,torch.roll(sam, shifts=(1, 1), dims=(2, 3))   
  ], axis = 1)
  
 

  return (sam == surrounding_pixels)[:,:,1:-1,1:-1]

In [5]:
def gradient_difs(output, target):
    #roll
    out_roll_2 = torch.roll(output,1,2)
    out_roll_3 = torch.roll(output,1,3)
    tar_roll_2 = torch.roll(target,1,2)
    tar_roll_3 = torch.roll(target,1,3)
    
    #get gradients
    out_grad_2 = output - out_roll_2
    out_grad_3 = output - out_roll_3
    tar_grad_2 = target - tar_roll_2
    tar_grad_3 = target - tar_roll_3
    
    #gradient diffs
    grad_2_dif = (out_grad_2 - tar_grad_2)**2
    grad_3_dif = (out_grad_3 - tar_grad_3)**2
    
    grad_dif = torch.mean(grad_2_dif + grad_3_dif)
    
    return grad_dif

In [6]:
## a pretty function, but it doesn't work :(
    ##our model doesn't do exactly half upscaling because the dimensions aren't powers of 2. using resize instead

def reduce_tensor(input_tensor, scale_down = 2, mode = "mean"):
    """
    mode can be mean, max, or min
    """
    batch, _, input_w, input_h = input_tensor.shape
    output_w = int(input_w/scale_down)
    output_h = int(input_h/scale_down)

    output_tensor = input_tensor.view(batch,scale_down**2,output_w,output_h)
    
    if mode == "mean":
      output_tensor = output_tensor.mean(dim=1)

    #if mode == "min":
     # output_tensor = torch.min(output_tensor,1)

    #if mode == "max":
     # output_tensor = output_tensor.max(dim=1)
    
    output_tensor = output_tensor.unsqueeze(1)
    
    return output_tensor

In [7]:
class MonocularDepthDataset(Dataset):
    def __init__(self, df, in_transform=None,out_transform = None ):
        self.df = df
        self.in_transform = in_transform
        self.out_transform = out_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = self.df.iloc[idx]['image']
        depth_path = self.df.iloc[idx]['depth']

        image = Image.open(image_path)
        depth = Image.open(depth_path)
        
        sam_path = self.df.iloc[idx]['image']
        sam_path = sam_path.split('/')
        sam_path[0] = 'sam_outputs/mask/'
        sam_path = ''.join(sam_path)
        sam = Image.open(sam_path)
        

        #torch.Size([1, 1, 2200, 1550])
        #torch.Size([1, 1, 1098, 773]) #d_0
        #torch.Size([1, 1, 550, 388]) #d_1
        #torch.Size([1, 1, 275, 194]) #d_2
        #torch.Size([1, 1, 138, 97]) #d_3
        #torch.Size([1, 1, 69, 49]) #d_4
        d_0 = depth.resize((773, 1098))#((1098, 773))#(depth)
        d_1 = depth.resize((388, 550))#((550, 388))#(depth)
        d_2 = depth.resize((194, 275))#((275, 194))#(depth)
        d_3 = depth.resize((97, 138))#((138, 97))#(depth)
        d_4 = depth.resize((49,69))#((69, 49))#(depth)



        if self.in_transform:
            image = self.in_transform(image)
            sam = self.in_transform(sam)
        if self.out_transform:
            depth = self.out_transform(depth)
            to_tensor = transforms.ToTensor()
            d_0 = to_tensor(d_0)
            d_1 = to_tensor(d_1)
            d_2 = to_tensor(d_2)
            d_3 = to_tensor(d_3)
            d_4 = to_tensor(d_4)
        #print(depth_path,depth.shape)
        #print('funct shape: ', sam.shape)
        #sam = neighbor_match(sam)
            
        return image, depth, d_0, d_1, d_2, d_3, d_4, sam

In [8]:
def conv_relu_block(in_channel,out_channel,kernel,padding):
    return nn.Sequential(
            nn.Conv2d(in_channel,out_channel, kernel_size = kernel, padding=padding),
            nn.ReLU()) #nn.ReLU(inplace=True) #nn.Ge

In [9]:
class vanilla_unet_early_exit(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        self.input_1 = conv_relu_block(3,3,3,1) ##grayscale inputs
        #self.input_2 = conv_relu_block(64, 64, 3, 1) #no extra channels
        
        self.samconv = conv_relu_block(8,2,1,0) #this has to match samid below, unless it is zero index? not sure
        
        self.base_model = models.resnet18(pretrained=True)
        self.base_layers = list(self.base_model.children())

        self.l0 = nn.Sequential(*self.base_layers[:3])
        self.U0_conv = conv_relu_block(64, 64, 1, 0)
        self.conv_up0 = conv_relu_block(64 + 256, 128, 3, 0)

        self.l1 = nn.Sequential(*self.base_layers[3:5])
        self.U1_conv = conv_relu_block(64, 64, 1, 0)
        self.conv_up1 = conv_relu_block(64 + 256, 256, 3, 1)

        self.l2 = self.base_layers[5]
        self.U2_conv = conv_relu_block(128, 128, 1, 0)
        self.conv_up2 = conv_relu_block(128 + 512, 256, 3, 1)

        self.l3 = self.base_layers[6]
        self.U3_conv = conv_relu_block(256, 256, 1, 0)
        self.conv_up3 = conv_relu_block(256 + 512, 512, 3, 1)

        self.l4 = self.base_layers[7]
        self.U4_conv = conv_relu_block(512, 512, 1, 0)

        self.conv_up4 = conv_relu_block(64 + 128, 64, 3, 1)

        self.out4 = nn.Conv2d(128, n_class, 1)

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x, sam):
        #max_sam_id = 100
        #print('starting shape: ', sam.shape)
        #sam = torch.nn.functional.one_hot(sam.to(torch.int64), num_classes= max_sam_id)
        #sam = torch.moveaxis(sam,4,2)
        #sam = torch.squeeze(sam,1) #now it is b x max_sam_id x h x w

        sam = self.samconv(sam.float())
        
        x = torch.cat([x,sam], axis = 1)
        x = self.input_1(x)
        
        #print(x.shape,'x')
         #concat on channel
        #x_one = self.input_2(x_one)
        block0 = self.l0(x)
        block1 = self.l1(block0)
        block2 = self.l2(block1)
        block3 = self.l3(block2)
        block4 = self.l4(block3)

        block4 = self.U4_conv(block4)
        b4_out = block4[:,-1,:,:].unsqueeze(1)
        #print(b4_out.shape, "block_4")
        x = nn.Upsample(size = (138,97), mode='bilinear', align_corners=True)(block4)
        block3 = self.U3_conv(block3)
        
        x = torch.cat([x, block3], axis=1)
        
        x = self.conv_up3(x)

        b3_out = x[:,-1,:,:].unsqueeze(1)
        #print(b3_out.shape, "block3")
        x = nn.Upsample(size = (275,194), mode='bilinear', align_corners=True)(x)
        
        block2 = self.U2_conv(block2)
        
        #print(x.shape, block2.shape)
        x = torch.cat([x, block2], axis=1)
        
        x = self.conv_up2(x)
        b2_out = x[:,-1,:,:].unsqueeze(1)
        #print(b2_out.shape, "block2")
        x = nn.Upsample(size = (550,388), mode='bilinear', align_corners=True)(x)
        block1 = self.U1_conv(block1)
        #print(x.shape, block1.shape)
        
        x = torch.cat([x, block1], axis=1)
        
        x = self.conv_up1(x)
        b1_out = x[:,-1,:,:].unsqueeze(1)
        #print(b1_out.shape, "block1")
        x = nn.Upsample(size = (1100, 775), mode='bilinear', align_corners=True)(x) 
        block0 = self.U0_conv(block0)

        #print(x.shape, block0.shape)
        x = torch.cat([x, block0], axis=1)

        
        x = self.conv_up0(x)
        b0_out = x[:,-1,:,:].unsqueeze(1)
        #print(b0_out.shape, "block0")
        out4 = self.out4(x)

        out4_upsampled = F.interpolate(out4, size=OUT_SIZE, mode='bilinear', align_corners=True)
        
        out = out4_upsampled
        
        
        return out, b0_out, b1_out, b2_out, b3_out, b4_out

In [10]:

in_path = "../../jkoh/inputs/"
y_path = '../../jkoh/depth_annotations/'
val_path = '../../krishna/project/val.csv'

class MonocularDepthDataset(Dataset):
    def __init__(self, df, in_transform=None,out_transform = None ):
        self.df = df
        self.in_transform = in_transform
        self.out_transform = out_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = self.df.iloc[idx]['image']
        depth_path = self.df.iloc[idx]['depth']

        image = Image.open(in_path + image_path)
        depth = Image.open(y_path + depth_path)
        
        sam_path = self.df.iloc[idx]['image']
        sam_path = '../../krishna/project/sam_outputs/val_mask/'+ sam_path
        sam = Image.open(sam_path)
        

        #torch.Size([1, 1, 2200, 1550])
        #torch.Size([1, 1, 1098, 773]) #d_0
        #torch.Size([1, 1, 550, 388]) #d_1
        #torch.Size([1, 1, 275, 194]) #d_2
        #torch.Size([1, 1, 138, 97]) #d_3
        #torch.Size([1, 1, 69, 49]) #d_4
        d_0 = depth.resize((773, 1098))#((1098, 773))#(depth)
        d_1 = depth.resize((388, 550))#((550, 388))#(depth)
        d_2 = depth.resize((194, 275))#((275, 194))#(depth)
        d_3 = depth.resize((97, 138))#((138, 97))#(depth)
        d_4 = depth.resize((49,69))#((69, 49))#(depth)



        if self.in_transform:
            image = self.in_transform(image)
            sam = self.in_transform(sam)
        if self.out_transform:
            depth = self.out_transform(depth)
            to_tensor = transforms.ToTensor()
            d_0 = to_tensor(d_0)
            d_1 = to_tensor(d_1)
            d_2 = to_tensor(d_2)
            d_3 = to_tensor(d_3)
            d_4 = to_tensor(d_4)
        #print(depth_path,depth.shape)
        #print('funct shape: ', sam.shape)
        #sam = neighbor_match(sam)
            
        return image, depth, d_0, d_1, d_2, d_3, d_4, sam
    
    
device = 'cuda' 

model = vanilla_unet_early_exit(n_class=1)
model = nn.DataParallel(model)
model.load_state_dict(torch.load('early_exit_final_final_40.pt'))
model = model.to(device)

#torch.device("cuda" if torch.cuda.is_available() else "cpu")

in_transform = transforms.Compose([
    transforms.Resize(OUT_SIZE),
    transforms.ToTensor()
])

out_transform = transforms.Compose([
    transforms.Resize(OUT_SIZE),
    transforms.ToTensor()
])

df = pd.read_csv(val_path)
test_dataset = MonocularDepthDataset(df, in_transform = in_transform, out_transform = out_transform)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model.eval()
criterion = SILogLoss()


test_loss = 0.0
metric = 0

with torch.no_grad():
    for images, depths, d_0, d_1, d_2, d_3, d_4, sam in tqdm.notebook.tqdm(test_dataloader):
        images = images.to(device)
        depths = depths.to(device)
        sam = neighbor_match(sam)
        sam = sam.to(device)
        d_0 = d_0.to(device)
        d_1 = d_1.to(device)
        d_2 = d_2.to(device)
        d_3 = d_3.to(device)
        d_4 = d_4.to(device)

        out, b0_out, b1_out, b2_out, b3_out, b4_out = model(images, sam)
        loss_pred = criterion(out, depths) ##output,targets
        loss_grad = gradient_difs(out, depths)
        loss_0 = criterion(b0_out, d_0)
        loss_1 = criterion(b1_out, d_1)
        loss_2 = criterion(b2_out, d_2)
        loss_3 = criterion(b3_out, d_3)
        loss_4 = criterion(b4_out, d_4)
        loss = loss_pred + loss_0 + loss_1 + loss_2 + loss_3 + loss_4 + loss_grad**0.5

        test_loss += loss.item()
        metric+=criterion(out,depths)
        #print(metric,loss_0)
        

metric /= len(test_dataloader)
print('Test Loss: ', metric)


/share/pkg.7/pytorch/1.13.1/install/lib/SCC/../python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/share/pkg.7/pytorch/1.13.1/install/lib/SCC/../python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


  0%|          | 0/270 [00:00<?, ?it/s]

Test Loss:  tensor(69.5564, device='cuda:0')


In [11]:
class vanilla_unet_early_exit(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        self.input_1 = conv_relu_block(3,3,3,1) ##grayscale inputs
        #self.input_2 = conv_relu_block(64, 64, 3, 1) #no extra channels
        
        self.samconv = conv_relu_block(50,2,1,0) #this has to match samid below, unless it is zero index? not sure
        
        self.base_model = models.resnet18(pretrained=True)
        self.base_layers = list(self.base_model.children())

        self.l0 = nn.Sequential(*self.base_layers[:3])
        self.U0_conv = conv_relu_block(64, 64, 1, 0)
        self.conv_up0 = conv_relu_block(64 + 256, 128, 3, 0)

        self.l1 = nn.Sequential(*self.base_layers[3:5])
        self.U1_conv = conv_relu_block(64, 64, 1, 0)
        self.conv_up1 = conv_relu_block(64 + 256, 256, 3, 1)

        self.l2 = self.base_layers[5]
        self.U2_conv = conv_relu_block(128, 128, 1, 0)
        self.conv_up2 = conv_relu_block(128 + 512, 256, 3, 1)

        self.l3 = self.base_layers[6]
        self.U3_conv = conv_relu_block(256, 256, 1, 0)
        self.conv_up3 = conv_relu_block(256 + 512, 512, 3, 1)

        self.l4 = self.base_layers[7]
        self.U4_conv = conv_relu_block(512, 512, 1, 0)

        self.conv_up4 = conv_relu_block(64 + 128, 64, 3, 1)

        self.out4 = nn.Conv2d(128, n_class, 1)

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x, sam):
        max_sam_id = 50
        #print('starting shape: ', sam.shape)
        sam = torch.nn.functional.one_hot(sam.to(torch.int64), num_classes= max_sam_id)
        sam = torch.moveaxis(sam,4,2)
        sam = torch.squeeze(sam,1) #now it is b x max_sam_id x h x w

        sam = self.samconv(sam.float())
        
        x = torch.cat([x,sam], axis = 1)
        x = self.input_1(x)
        
        #print(x.shape,'x')
         #concat on channel
        #x_one = self.input_2(x_one)
        block0 = self.l0(x)
        block1 = self.l1(block0)
        block2 = self.l2(block1)
        block3 = self.l3(block2)
        block4 = self.l4(block3)

        block4 = self.U4_conv(block4)
        b4_out = block4[:,-1,:,:].unsqueeze(1)
        #print(b4_out.shape, "block_4")
        x = nn.Upsample(size = (138,97), mode='bilinear', align_corners=True)(block4)
        block3 = self.U3_conv(block3)
        
        x = torch.cat([x, block3], axis=1)
        
        x = self.conv_up3(x)

        b3_out = x[:,-1,:,:].unsqueeze(1)
        #print(b3_out.shape, "block3")
        x = nn.Upsample(size = (275,194), mode='bilinear', align_corners=True)(x)
        
        block2 = self.U2_conv(block2)
        
        #print(x.shape, block2.shape)
        x = torch.cat([x, block2], axis=1)
        
        x = self.conv_up2(x)
        b2_out = x[:,-1,:,:].unsqueeze(1)
        #print(b2_out.shape, "block2")
        x = nn.Upsample(size = (550,388), mode='bilinear', align_corners=True)(x)
        block1 = self.U1_conv(block1)
        #print(x.shape, block1.shape)
        
        x = torch.cat([x, block1], axis=1)
        
        x = self.conv_up1(x)
        b1_out = x[:,-1,:,:].unsqueeze(1)
        #print(b1_out.shape, "block1")
        x = nn.Upsample(size = (1100, 775), mode='bilinear', align_corners=True)(x) 
        block0 = self.U0_conv(block0)

        #print(x.shape, block0.shape)
        x = torch.cat([x, block0], axis=1)

        
        x = self.conv_up0(x)
        b0_out = x[:,-1,:,:].unsqueeze(1)
        #print(b0_out.shape, "block0")
        out4 = self.out4(x)

        out4_upsampled = F.interpolate(out4, size=OUT_SIZE, mode='bilinear', align_corners=True)
        
        out = out4_upsampled
        
        
        return out, b0_out, b1_out, b2_out, b3_out, b4_out

In [12]:

in_path = "../../jkoh/inputs/"
y_path = '../../jkoh/depth_annotations/'
val_path = '../../krishna/project/val.csv'

class MonocularDepthDataset(Dataset):
    def __init__(self, df, in_transform=None,out_transform = None ):
        self.df = df
        self.in_transform = in_transform
        self.out_transform = out_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = self.df.iloc[idx]['image']
        depth_path = self.df.iloc[idx]['depth']

        image = Image.open(in_path + image_path)
        depth = Image.open(y_path + depth_path)
        
        sam_path = self.df.iloc[idx]['image']
        sam_path = '../../krishna/project/sam_outputs/val_mask/'+ sam_path
        sam = Image.open(sam_path)
        

        #torch.Size([1, 1, 2200, 1550])
        #torch.Size([1, 1, 1098, 773]) #d_0
        #torch.Size([1, 1, 550, 388]) #d_1
        #torch.Size([1, 1, 275, 194]) #d_2
        #torch.Size([1, 1, 138, 97]) #d_3
        #torch.Size([1, 1, 69, 49]) #d_4
        d_0 = depth.resize((773, 1098))#((1098, 773))#(depth)
        d_1 = depth.resize((388, 550))#((550, 388))#(depth)
        d_2 = depth.resize((194, 275))#((275, 194))#(depth)
        d_3 = depth.resize((97, 138))#((138, 97))#(depth)
        d_4 = depth.resize((49,69))#((69, 49))#(depth)



        if self.in_transform:
            image = self.in_transform(image)
            sam = self.in_transform(sam)
        if self.out_transform:
            depth = self.out_transform(depth)
            to_tensor = transforms.ToTensor()
            d_0 = to_tensor(d_0)
            d_1 = to_tensor(d_1)
            d_2 = to_tensor(d_2)
            d_3 = to_tensor(d_3)
            d_4 = to_tensor(d_4)
        #print(depth_path,depth.shape)
        #print('funct shape: ', sam.shape)
        #sam = neighbor_match(sam)
            
        return image, depth, d_0, d_1, d_2, d_3, d_4, sam
    
    
device = 'cuda' 

model = vanilla_unet_early_exit(n_class=1)
model = nn.DataParallel(model)
model.load_state_dict(torch.load('early_exit_unet_sam_grad_diffs.pt'))
#model.load_state_dict(torch.load('early_exit_final_final_40.pt'))
model = model.to(device)

#torch.device("cuda" if torch.cuda.is_available() else "cpu")

in_transform = transforms.Compose([
    transforms.Resize(OUT_SIZE),
    transforms.ToTensor()
])

out_transform = transforms.Compose([
    transforms.Resize(OUT_SIZE),
    transforms.ToTensor()
])

df = pd.read_csv(val_path)
test_dataset = MonocularDepthDataset(df, in_transform = in_transform, out_transform = out_transform)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model.eval()
criterion = SILogLoss()


test_loss = 0.0
metric = 0

with torch.no_grad():
    for images, depths, d_0, d_1, d_2, d_3, d_4, sam in tqdm.notebook.tqdm(test_dataloader):
        images = images.to(device)
        depths = depths.to(device)
        #sam = neighbor_match(sam)
        sam = sam.to(device)
        d_0 = d_0.to(device)
        d_1 = d_1.to(device)
        d_2 = d_2.to(device)
        d_3 = d_3.to(device)
        d_4 = d_4.to(device)

        out, b0_out, b1_out, b2_out, b3_out, b4_out = model(images, sam)
        loss_pred = criterion(out, depths) ##output,targets
        loss_grad = gradient_difs(out, depths)
        loss_0 = criterion(b0_out, d_0)
        loss_1 = criterion(b1_out, d_1)
        loss_2 = criterion(b2_out, d_2)
        loss_3 = criterion(b3_out, d_3)
        loss_4 = criterion(b4_out, d_4)
        loss = loss_pred + loss_0 + loss_1 + loss_2 + loss_3 + loss_4 + loss_grad**0.5

        test_loss += loss.item()
        metric+=criterion(out,depths)
        #print(metric,loss_0)
        

metric /= len(test_dataloader)
print('Test Loss: ', metric)


/share/pkg.7/pytorch/1.13.1/install/lib/SCC/../python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/share/pkg.7/pytorch/1.13.1/install/lib/SCC/../python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


  0%|          | 0/270 [00:00<?, ?it/s]

Test Loss:  tensor(69.5564, device='cuda:0')
